In [1]:
import os
from pytorch_lightning import loggers as pl_loggers
import pytorch_lightning as pl
from image_inpainting.datamodule.image_net_data_module import ImageNetDataModule
from pytorch_lightning.callbacks import ModelCheckpoint
from image_inpainting.model.context_encoder import ContextEncoder
from image_inpainting.utils import print_results_images

In [2]:
from datetime import datetime
now = datetime.now()
now = now.strftime("%Y-%m-%d_%H-%M-%S")

## Create datamodule

In [3]:
data_dir = "data"

dm = ImageNetDataModule(
    data_dir=os.path.join(data_dir, "imagenet-128"), 
    batch_size_train=512,
    batch_size_val=512,
    batch_size_test=512,
    num_workers=10, 
    pin_memory=True, 
    persistent_workers=True
)

## Create a ContextEncoder model from scratch

In [4]:
model = ContextEncoder(input_size=(3, 128, 128), hidden_size=4000, save_image_per_epoch=True)

## Or load it from a checkpoint

In [4]:
model = ContextEncoder.load_from_checkpoint("checkpoints/imagenet_128/2024-12-17_17-07-56-epoch=42-val_loss=0.42.ckpt") # change the path to your checkpoint
model.enable_save_image_per_epoch()
model.to("cuda")

ContextEncoder(
  (psnr_metric): PeakSignalNoiseRatio()
  (joint_loss): JointLoss(
    (rec_loss): ReconstructionLoss()
    (adv_loss): AdversarialLoss(
      (loss_function): BCELoss()
    )
  )
  (rec_loss): ReconstructionLoss()
  (adv_loss): AdversarialLoss(
    (loss_function): BCELoss()
  )
  (generator): AdversarialGenerator(
    (encoder): Encoder(
      (encoder): Sequential(
        (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2, inplace=True)
        (2): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): LeakyReLU(negative_slope=0.2, inplace=True)
        (5): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (7): LeakyReLU(negative_slope=0.2, inplace=True)
        (8): Conv2

## Train it

In [5]:
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/imagenet_128',
    filename=now+'-{epoch:02d}-{val_loss:.2f}',
    monitor='val_loss',
    save_top_k=-1,  # Save all checkpoints
    every_n_epochs=1  # Save checkpoint every n epochs
)

tb_logger = pl_loggers.TensorBoardLogger("Context_Encoder_Inpainting")
trainer = pl.Trainer(max_epochs=300, devices=-1, accelerator="cuda", logger=tb_logger, callbacks=[checkpoint_callback])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
# trainer.fit(model, dm)

# To continue training:
trainer.fit(model, dm, ckpt_path="checkpoints/imagenet_128/2024-12-17_08-44-43-epoch=38-val_loss=0.42.ckpt")

You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Note: The ImageNet dataset can't be downloaded automatically. Please refer to the README if you haven't already downloaded it. Otherwise you can skip this message.


d:\Programs\Anaconda\envs\image_inpainting\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\CYTechNVME\AIIP_Projet\Image-Inpainting-with-Basic-Convolutional-Networks\checkpoints\imagenet_128 exists and is not empty.
Restoring states from the checkpoint path at checkpoints/imagenet_128/2024-12-17_08-44-43-epoch=38-val_loss=0.42.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                     | Params | Mode 
-------------------------------------------------------------------
0 | psnr_metric   | PeakSignalNoiseRatio     | 0      | train
1 | joint_loss    | JointLoss                | 0      | train
2 | rec_loss      | ReconstructionLoss       | 0      | train
3 | adv_loss      | AdversarialLoss          | 0      | train
4 | generator     | AdversarialGenerator     | 71.1 M | train
5 | discriminator | AdversarialDiscriminator | 2.8 M  | train
-------------------------------------------------------------------
73.9 M 

Epoch 43:   9%|▉         | 211/2306 [01:55<19:06,  1.83it/s, v_num=2, train_psnr=18.00, train_loss_context_encoder=0.216, train_loss_discriminator=2.46e-14, val_psnr=14.70, val_loss=0.419] 


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

A notebook "tensorboard" exists if you want to check how the metrics evolve during training

## Results on the last epoch

Here the results of this cell are after ????????? epochs on the full Image Net dataset cropped and resized to 128x128.

- **Number of steps**: 
- **Time**: 
- **Observations (with tensorboard)**: 

In [ ]:
trainer.test(model, dm)

In [ ]:
dm.setup("test") # in case "test" wasn't called before this cell

x, y = next(iter(dm.test_dataloader()))
x = x.to(model.device)
y = y.to(model.device)

out = model.forward(x)

print_results_images(x, y, out, "Results on test set", dm.inverse_transform)

In [ ]:
dm.setup("fit") # in case "fit" wasn't called before this cell

x, y = next(iter(dm.train_dataloader()))
x = x.to(model.device)
y = y.to(model.device)
out = model.forward(x)

print_results_images(x, y, out, "Results on training set", dm.inverse_transform)